In [9]:
# IMPORT LIBRARIES

import numpy as np
import pandas as pd

In [14]:
"""
Assumes:
- Time invariance (uses survival and fertility rates from one year to project entirety of Focal's lifetime)
- If migration is used: Immigration and emigration rates are equal
- All other assumptions from original time invariant DemoKin code
"""

class KinTimeInvariant:
    def __init__(self, p, f, mig=None, birth_female=1/2.04, pi=None, output_kin=None, include_migration=False):
        """
        Initialize the KinTimeInvariant class with the given parameters.

        Parameters:
        - p: Vector of survival probabilities.
        - f: Vector of fertility rates.
        - mig: Vector of migration rates.
        - birth_female: Fraction of births that are female.
        - pi: Stationary distribution vector (optional).
        - output_kin: List of kin types to be included in the output (optional).
        - include_migration: Boolean flag indicating whether to include the Rogers-Castro migration model (optional).
        """
        self.p = p
        self.f = f
        self.mig = mig if mig is not None else np.zeros(len(p))
        self.birth_female = birth_female
        self.pi = pi
        self.output_kin = output_kin
        self.include_migration = include_migration

        self.ages = len(p)
        self.age = np.arange(self.ages)

        # Initialize survival and fertility transition matrices
        self.Ut, self.Mt, self.Um, self.zeros = self.initialize_matrices()
        self.ft, self.e = self.initialize_fertility_matrix()

        # Calculate stationary distribution if not provided
        if self.pi is None:
            self.pi = self.calculate_stationary_distribution()

        # Initialize kin matrices
        self.kin_dict = self.initialize_kin_matrices()

        # Propagate kin over time
        if self.include_migration:
            self.propagate_kin_migrate()
        else:
            self.propagate_kin()

        # Update kin dictionary to align dead kin and remove unwanted kin types
        self.update_kin_dict()

    def initialize_matrices(self):
        Ut = np.zeros((self.ages, self.ages))
        Mt = np.zeros((self.ages, self.ages))
        Um = np.zeros((self.ages, self.ages))  # Migration transition matrix
        zeros = np.zeros((self.ages, self.ages))

        for i in range(self.ages - 1):
            Ut[i + 1, i] = self.p[i] * (1 - self.mig[i])
            Um[i + 1, i] = self.p[i] * self.mig[i]

        Ut[self.ages - 1, self.ages - 1] = self.p[self.ages - 1] * (1 - self.mig[self.ages - 1])
        Um[self.ages - 1, self.ages - 1] = self.p[self.ages - 1] * self.mig[self.ages - 1]

        np.fill_diagonal(Mt, 1 - np.array(self.p) * (1 - np.array(self.mig)))

        Ut = np.block([
            [Ut, zeros],
            [Mt, zeros]
        ])

        Um = np.block([
            [Um, zeros],
            [zeros, zeros]
        ])

        return Ut, Mt, Um, zeros

    def initialize_fertility_matrix(self):
        """
        Initialize the fertility matrix and identity matrix.

        Returns:
        - ft: Fertility matrix.
        - e: Identity matrix with appropriate dimensions.
        """
        ft = np.zeros((self.ages * 2, self.ages * 2))
        ft[0, :self.ages] = np.array(self.f) * self.birth_female

        e = np.zeros((self.ages * 2, self.ages * 2))
        np.fill_diagonal(e[:self.ages, :self.ages], 1)

        return ft, e

    def calculate_stationary_distribution(self):
        """
        Calculate the stationary distribution vector pi.

        Returns:
        - pi: Stationary distribution vector.
        Calculates and returns the largest eigenvector instead of the first one in the matrix.
        """
        # Compute matrix A
        A = self.Ut[0:self.ages, 0:self.ages] + self.ft[0:self.ages, 0:self.ages]
        # Compute eigenvalues and eigenvectors
        eigenvalues, eigenvectors = np.linalg.eig(A)
        # Calculate the magnitude (L2 norm) of each eigenvector
        magnitudes = np.linalg.norm(eigenvectors, axis=0)
        # Find the index of the largest eigenvector
        largest_eigenvector_index = np.argmax(magnitudes)
        # Get the largest eigenvector
        w = np.real(eigenvectors[:, largest_eigenvector_index])
        # Normalize the largest eigenvector
        w = w / np.sum(w)
        # Compute the stationary distribution pi
        pi = w * A[0, :] / np.sum(w * A[0, :])
        return pi.reshape(-1)

    def initialize_kin_matrices(self):
        """
        Initialize matrices for different kin types.

        Returns:
        - kin_dict: Dictionary containing initialized kin matrices.
        """
        d = np.zeros((self.ages * 2, self.ages))
        gd = np.zeros((self.ages * 2, self.ages))
        ggd = np.zeros((self.ages * 2, self.ages))
        m = np.zeros((self.ages * 2, self.ages))
        gm = np.zeros((self.ages * 2, self.ages))
        ggm = np.zeros((self.ages * 2, self.ages))
        os = np.zeros((self.ages * 2, self.ages))
        ys = np.zeros((self.ages * 2, self.ages))
        nos = np.zeros((self.ages * 2, self.ages))
        nys = np.zeros((self.ages * 2, self.ages))
        oa = np.zeros((self.ages * 2, self.ages))
        ya = np.zeros((self.ages * 2, self.ages))
        coa = np.zeros((self.ages * 2, self.ages))
        cya = np.zeros((self.ages * 2, self.ages))

        d_mig = np.zeros((self.ages * 2, self.ages))
        gd_mig = np.zeros((self.ages * 2, self.ages))
        ggd_mig = np.zeros((self.ages * 2, self.ages))
        m_mig = np.zeros((self.ages * 2, self.ages))
        gm_mig = np.zeros((self.ages * 2, self.ages))
        ggm_mig = np.zeros((self.ages * 2, self.ages))
        os_mig = np.zeros((self.ages * 2, self.ages))
        ys_mig = np.zeros((self.ages * 2, self.ages))
        nos_mig = np.zeros((self.ages * 2, self.ages))
        nys_mig = np.zeros((self.ages * 2, self.ages))
        oa_mig = np.zeros((self.ages * 2, self.ages))
        ya_mig = np.zeros((self.ages * 2, self.ages))
        coa_mig = np.zeros((self.ages * 2, self.ages))
        cya_mig = np.zeros((self.ages * 2, self.ages))

        # Initialize the mother matrix with the stationary distribution
        m[:len(self.pi), 0] = self.pi

        kin_dict = {
            'd': d, # daughters
            'gd': gd, # granddaughters
            'ggd': ggd, # great granddaughters
            'm': m, # mother
            'gm': gm, # grandmothers
            'ggm': ggm, # great grandmothers
            'os': os, # older sisters
            'ys': ys, # younger sisters
            'nos': nos, # nieces from older sisters
            'nys': nys, # nieces from younger sisters
            'oa': oa, # aunts older than mother
            'ya': ya, # aunts younger than mother
            'coa': coa, # cousins from older aunts
            'cya': cya, # cousins from younger aunts

            'd_mig': d_mig, # daughters
            'gd_mig': gd_mig, # granddaughters
            'ggd_mig': ggd_mig, # great granddaughters
            'm_mig': m_mig, # mother
            'gm_mig': gm_mig, # grandmothers
            'ggm_mig': ggm_mig, # great
            'os_mig': os_mig, # older sisters
            'ys_mig': ys_mig, # younger sisters
            'nos_mig': nos_mig, # maxnices from older sisters
            'nys_mig': nys_mig, # maxnices from younger sisters
            'oa_mig': oa_mig, # aunts older than mother
            'ya_mig': ya_mig, # aunts younger than mother
            'coa_mig': coa_mig, # cousins from older aunts
            'cya_mig': cya_mig # cousins from younger aunts
        }

        return kin_dict

    # Runs through Focal's lifetime to "propagate" the kin network. Note that
    # the formulas take the general form of k(i+1) = U*k(i) + f. Recall that Ut
    # is the survival rate transition matrix, and f is the fertility rate
    # transition matrix.
    # Using the migration rates from the input argument "mig", kin are removed
    # from the original matrix representing the home city to the migration matrix
    # representing the state of having moved away. The same migration rates are
    # then applied to the migration matrix, to determine the number of kin that
    # will move back to the home city.
    # The name of the matrix within kin_dict refers to the type of kin; see
    # initialize_kin_matrices() method for key
    def propagate_kin(self):
        """
        Propagate kin over time based on the initialized matrices.
        """
        for i in range(0, self.ages - 1):
            self.kin_dict['d'][:, i + 1] = np.dot(self.Ut, self.kin_dict['d'][:, i]) + np.dot(self.ft, self.e[:, i])
            self.kin_dict['gd'][:, i + 1] = np.dot(self.Ut, self.kin_dict['gd'][:, i]) + np.dot(self.ft, self.kin_dict['d'][:, i])
            self.kin_dict['ggd'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ggd'][:, i]) + np.dot(self.ft, self.kin_dict['gd'][:, i])
            self.kin_dict['m'][:, i + 1] = np.dot(self.Ut, self.kin_dict['m'][:, i])
            self.kin_dict['ys'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ys'][:, i]) + np.dot(self.ft, self.kin_dict['m'][:, i])
            self.kin_dict['nys'][:, i + 1] = np.dot(self.Ut, self.kin_dict['nys'][:, i]) + np.dot(self.ft, self.kin_dict['ys'][:, i])

        self.kin_dict['gm'][:self.ages, 0] = np.dot(self.kin_dict['m'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['gm'][:, i + 1] = np.dot(self.Ut, self.kin_dict['gm'][:, i])

        self.kin_dict['ggm'][:self.ages, 0] = np.dot(self.kin_dict['gm'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['ggm'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ggm'][:, i])

        self.kin_dict['os'][:self.ages, 0] = np.dot(self.kin_dict['d'][:self.ages, :], self.pi)
        self.kin_dict['nos'][:self.ages, 0] = np.dot(self.kin_dict['gd'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['os'][:, i + 1] = np.dot(self.Ut, self.kin_dict['os'][:, i])
            self.kin_dict['nos'][:, i + 1] = np.dot(self.Ut, self.kin_dict['nos'][:, i]) + np.dot(self.ft, self.kin_dict['os'][:, i])

        self.kin_dict['oa'][:self.ages, 0] = np.dot(self.kin_dict['os'][:self.ages, :], self.pi)
        self.kin_dict['ya'][:self.ages, 0] = np.dot(self.kin_dict['ys'][:self.ages, :], self.pi)
        self.kin_dict['coa'][:self.ages, 0] = np.dot(self.kin_dict['nos'][:self.ages, :], self.pi)
        self.kin_dict['cya'][:self.ages, 0] = np.dot(self.kin_dict['nys'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['oa'][:, i + 1] = np.dot(self.Ut, self.kin_dict['oa'][:, i])
            self.kin_dict['ya'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ya'][:, i]) + np.dot(self.ft, self.kin_dict['gm'][:, i])
            self.kin_dict['coa'][:, i + 1] = np.dot(self.Ut, self.kin_dict['coa'][:, i]) + np.dot(self.ft, self.kin_dict['oa'][:, i])
            self.kin_dict['cya'][:, i + 1] = np.dot(self.Ut, self.kin_dict['cya'][:, i]) + np.dot(self.ft, self.kin_dict['ya'][:, i])

    def propagate_kin_migrate(self):
        """
        Propagate kin over time based on the initialized matrices with migration and return migration. Note that migration transition matrices
        for both immigration and emigration are the same, demonstrating the assumption that migration rates are equal.
        """
        for i in range(0, self.ages - 1):
            # Update the original kin matrices with migration considered
            self.kin_dict['d'][:, i + 1] = np.dot(self.Ut, self.kin_dict['d'][:, i]) + np.dot(self.ft, self.e[:, i])
            self.kin_dict['d'][:, i + 1] -= np.dot(self.Um, self.kin_dict['d'][:, i])  # Decrease due to migration
            self.kin_dict['d'][:, i + 1] += np.dot(self.Um, self.kin_dict['d_mig'][:, i])  # Increase due to return migration

            self.kin_dict['gd'][:, i + 1] = np.dot(self.Ut, self.kin_dict['gd'][:, i]) + np.dot(self.ft, self.kin_dict['d'][:, i])
            self.kin_dict['gd'][:, i + 1] -= np.dot(self.Um, self.kin_dict['gd'][:, i])  # Decrease due to migration
            self.kin_dict['gd'][:, i + 1] += np.dot(self.Um, self.kin_dict['gd_mig'][:, i])  # Increase due to return migration

            self.kin_dict['ggd'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ggd'][:, i]) + np.dot(self.ft, self.kin_dict['gd'][:, i])
            self.kin_dict['ggd'][:, i + 1] -= np.dot(self.Um, self.kin_dict['ggd'][:, i])  # Decrease due to migration
            self.kin_dict['ggd'][:, i + 1] += np.dot(self.Um, self.kin_dict['ggd_mig'][:, i])  # Increase due to return migration

            self.kin_dict['m'][:, i + 1] = np.dot(self.Ut, self.kin_dict['m'][:, i])
            self.kin_dict['m'][:, i + 1] -= np.dot(self.Um, self.kin_dict['m'][:, i])  # Decrease due to migration
            self.kin_dict['m'][:, i + 1] += np.dot(self.Um, self.kin_dict['m_mig'][:, i])  # Increase due to return migration

            self.kin_dict['ys'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ys'][:, i]) + np.dot(self.ft, self.kin_dict['m'][:, i])
            self.kin_dict['ys'][:, i + 1] -= np.dot(self.Um, self.kin_dict['ys'][:, i])  # Decrease due to migration
            self.kin_dict['ys'][:, i + 1] += np.dot(self.Um, self.kin_dict['ys_mig'][:, i])  # Increase due to return migration

            self.kin_dict['nys'][:, i + 1] = np.dot(self.Ut, self.kin_dict['nys'][:, i]) + np.dot(self.ft, self.kin_dict['ys'][:, i])
            self.kin_dict['nys'][:, i + 1] -= np.dot(self.Um, self.kin_dict['nys'][:, i])  # Decrease due to migration
            self.kin_dict['nys'][:, i + 1] += np.dot(self.Um, self.kin_dict['nys_mig'][:, i])  # Increase due to return migration

            # Update migrated kin matrices
            self.kin_dict['d_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['d'][:, i]) + np.dot(self.Ut, self.kin_dict['d_mig'][:, i])
            self.kin_dict['gd_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['gd'][:, i]) + np.dot(self.Ut, self.kin_dict['gd_mig'][:, i])
            self.kin_dict['ggd_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['ggd'][:, i]) + np.dot(self.Ut, self.kin_dict['ggd_mig'][:, i])
            self.kin_dict['m_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['m'][:, i]) + np.dot(self.Ut, self.kin_dict['m_mig'][:, i])
            self.kin_dict['ys_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['ys'][:, i]) + np.dot(self.Ut, self.kin_dict['ys_mig'][:, i])
            self.kin_dict['nys_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['nys'][:, i]) + np.dot(self.Ut, self.kin_dict['nys_mig'][:, i])

        self.kin_dict['gm'][:self.ages, 0] = np.dot(self.kin_dict['m'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['gm'][:, i + 1] = np.dot(self.Ut, self.kin_dict['gm'][:, i])

        self.kin_dict['ggm'][:self.ages, 0] = np.dot(self.kin_dict['gm'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['ggm'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ggm'][:, i])

        self.kin_dict['os'][:self.ages, 0] = np.dot(self.kin_dict['d'][:self.ages, :], self.pi)
        self.kin_dict['nos'][:self.ages, 0] = np.dot(self.kin_dict['gd'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['os'][:, i + 1] = np.dot(self.Ut, self.kin_dict['os'][:, i])
            self.kin_dict['os'][:, i + 1] -= np.dot(self.Um, self.kin_dict['os'][:, i])  # Decrease due to migration
            self.kin_dict['os'][:, i + 1] += np.dot(self.Um, self.kin_dict['os_mig'][:, i])  # Increase due to return migration

            self.kin_dict['nos'][:, i + 1] = np.dot(self.Ut, self.kin_dict['nos'][:, i]) + np.dot(self.ft, self.kin_dict['os'][:, i])
            self.kin_dict['nos'][:, i + 1] -= np.dot(self.Um, self.kin_dict['nos'][:, i])  # Decrease due to migration
            self.kin_dict['nos'][:, i + 1] += np.dot(self.Um, self.kin_dict['nos_mig'][:, i])  # Increase due to return migration

            # Update migrated kin matrices for older sisters and nieces from older sisters
            self.kin_dict['os_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['os'][:, i]) + np.dot(self.Ut, self.kin_dict['os_mig'][:, i])
            self.kin_dict['nos_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['nos'][:, i]) + np.dot(self.Ut, self.kin_dict['nos_mig'][:, i]) + np.dot(self.ft, self.kin_dict['os_mig'][:, i])

        self.kin_dict['oa'][:self.ages, 0] = np.dot(self.kin_dict['os'][:self.ages, :], self.pi)
        self.kin_dict['ya'][:self.ages, 0] = np.dot(self.kin_dict['ys'][:self.ages, :], self.pi)
        self.kin_dict['coa'][:self.ages, 0] = np.dot(self.kin_dict['nos'][:self.ages, :], self.pi)
        self.kin_dict['cya'][:self.ages, 0] = np.dot(self.kin_dict['nys'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['oa'][:, i + 1] = np.dot(self.Ut, self.kin_dict['oa'][:, i])
            self.kin_dict['oa'][:, i + 1] -= np.dot(self.Um, self.kin_dict['oa'][:, i])  # Decrease due to migration
            self.kin_dict['oa'][:, i + 1] += np.dot(self.Um, self.kin_dict['oa_mig'][:, i])  # Increase due to return migration

            self.kin_dict['ya'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ya'][:, i]) + np.dot(self.ft, self.kin_dict['gm'][:, i])
            self.kin_dict['ya'][:, i + 1] -= np.dot(self.Um, self.kin_dict['ya'][:, i])  # Decrease due to migration
            self.kin_dict['ya'][:, i + 1] += np.dot(self.Um, self.kin_dict['ya_mig'][:, i])  # Increase due to return migration

            self.kin_dict['coa'][:, i + 1] = np.dot(self.Ut, self.kin_dict['coa'][:, i]) + np.dot(self.ft, self.kin_dict['oa'][:, i])
            self.kin_dict['coa'][:, i + 1] -= np.dot(self.Um, self.kin_dict['coa'][:, i])  # Decrease due to migration
            self.kin_dict['coa'][:, i + 1] += np.dot(self.Um, self.kin_dict['coa_mig'][:, i])  # Increase due to return migration

            self.kin_dict['cya'][:, i + 1] = np.dot(self.Ut, self.kin_dict['cya'][:, i]) + np.dot(self.ft, self.kin_dict['ya'][:, i])
            self.kin_dict['cya'][:, i + 1] -= np.dot(self.Um, self.kin_dict['cya'][:, i])  # Decrease due to migration
            self.kin_dict['cya'][:, i + 1] += np.dot(self.Um, self.kin_dict['cya_mig'][:, i])  # Increase due to return migration

            # Update migrated kin matrices for aunts and cousins
            self.kin_dict['oa_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['oa'][:, i]) + np.dot(self.Ut, self.kin_dict['oa_mig'][:, i])
            self.kin_dict['ya_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['ya'][:, i]) + np.dot(self.Ut, self.kin_dict['ya_mig'][:, i]) + np.dot(self.ft, self.kin_dict['gm_mig'][:, i])
            self.kin_dict['coa_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['coa'][:, i]) + np.dot(self.Ut, self.kin_dict['coa_mig'][:, i]) + np.dot(self.ft, self.kin_dict['oa_mig'][:, i])
            self.kin_dict['cya_mig'][:, i + 1] = np.dot(self.Um, self.kin_dict['cya'][:, i]) + np.dot(self.Ut, self.kin_dict['cya_mig'][:, i]) + np.dot(self.ft, self.kin_dict['ya_mig'][:, i])


    def update_kin_dict(self):
        """
        Update the kin dictionary by removing unwanted kin types and aligning dead kin matrices.
        """
        if self.output_kin is not None:
            for key in list(self.kin_dict.keys()):
                if key not in self.output_kin:
                    del self.kin_dict[key]

        for key, matrix in self.kin_dict.items():
            matrix[self.ages:2*self.ages, :self.ages-1] = matrix[self.ages:2*self.ages, 1:self.ages]
            matrix[self.ages:2*self.ages, self.ages-1] = 0
            self.kin_dict[key] = matrix

    def create_output_dataframe(self):
        """
        Create an output DataFrame from the kin matrices.

        Returns:
        - output: DataFrame containing kin information.
        """
        output = pd.DataFrame()

        for kin_name, matrix in self.kin_dict.items():
            if kin_name.endswith('_mig'):
                continue  # Skip migrated kin matrices

            kin_type = []
            kin_ages = []
            focal_ages = []
            living_kin = []
            dead_kin = []
            migrated_kin = []

            for kin_age in range(self.ages):
                for focal_age in range(self.ages):
                    kin_type.append(kin_name)
                    kin_ages.append(kin_age)
                    focal_ages.append(focal_age)
                    living_kin.append(matrix[kin_age, focal_age])
                    dead_kin.append(matrix[kin_age + self.ages, focal_age])
                    if self.include_migration:
                        migrated_kin.append(self.kin_dict[kin_name + '_mig'][kin_age, focal_age])

            if self.include_migration:
                kin_df = pd.DataFrame({
                    "Kin": kin_type,
                    'Age of Kin': kin_ages,
                    'Age of Focal': focal_ages,
                    'Number of Living Kin': living_kin,
                    'Number of Dead Kin': dead_kin,
                    'Number of Living Kin Migrated Away': migrated_kin
                })
            else:
                kin_df = pd.DataFrame({
                    "Kin": kin_type,
                    'Age of Kin': kin_ages,
                    'Age of Focal': focal_ages,
                    'Number of Living Kin': living_kin,
                    'Number of Dead Kin': dead_kin
                })

            output = pd.concat([output, kin_df], ignore_index=True)

        return output

    def run(self):
        """
        Run the kin propagation model and return the output DataFrame.

        Returns:
        - output: DataFrame containing kin information.
        """
        return self.create_output_dataframe()


In [13]:
# RUN AND PRINT RESULTS

# Run without Rogers-Castro migration model
swe_surv_df = pd.read_csv('swe_surv_2015.csv', names=['age', 'surv_prob'], skiprows=1)
swe_fert_df = pd.read_csv('swe_asfr_2015.csv', names=['age', 'fert_prob'], skiprows=1)

p = swe_surv_df['surv_prob'].to_list()
f = swe_fert_df['fert_prob'].to_list()

kin_model = KinTimeInvariant(p, f)
result = kin_model.run()

result.to_csv('output.csv', index = False)


# Run with Rogers Castro migration model
migration_rates = pd.read_csv('migration_rates.csv', names=['mig_prob'], skiprows=1)
m = migration_rates['mig_prob'].to_list()

kin_model_migration = KinTimeInvariant(p, f, mig=m, include_migration=True)
result_migration = kin_model_migration.run()

result_migration.to_csv('output_migration.csv', index = False)

<ipython-input-11-89fa0ecf3891>:110: RuntimeWarning: divide by zero encountered in divide
  pi = w * A[0, :] / np.sum(w * A[0, :])
<ipython-input-11-89fa0ecf3891>:110: RuntimeWarning: invalid value encountered in divide
  pi = w * A[0, :] / np.sum(w * A[0, :])
